In [1]:
import openpyxl
import re
import requests
import os
from sys import argv
from openpyxl import Workbook
from datetime import datetime
import html.parser
from pyquery import PyQuery as pq

In [2]:
def download_apk(url):
    """
    下载文件
    :param url:下载链接
    :param num:索引值
    """
    global count
    global successful_count
    count += 1
    succeed = 'Succeed'
    failure = 'Failure'
    print('第' + str(count) + '条url:\n' + url)
    filename = os.path.basename(url)
    filename = filename.split('?')[0]
    response = requests.head(url)
    filesize = round(float(response.headers['Content-Length']) / 1048576, 2)
    apk_format = 'application/vnd.android.package-archive'

    # 过滤非zip文件或大于100.00M的文件
    # TODO 可按需修改
    if response.headers['Content-Type'] == apk_format and filesize < 100.00:
        print('文件类型：' + response.headers['Content-Type'] + "\n" +
              '文件大小：' + str(filesize) + 'M' + "\n" +
              '文件名：' + str(filename))

        # 下载文件
        headers = {
            'User-Agent': 'Mozilla/5.0 (Linux; U; Android 4.0.3; zh-cn; M032 Build/IML74K) '
                          'AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 Mobile Safari/534.30',
            'Connection': 'keep-alive', }
        file = requests.get(url, headers=headers, timeout=10)

        with open(filename, 'wb') as apk:
            apk.write(file.content)
            print(succeed + "\n")
            successful_count += 1

    else:
        print('文件类型:' + response.headers['Content-Type'] + "\n" +
              '文件大小:' + str(filesize) + 'M' + "\n" +
              failure + "\n")

In [3]:
def crawl(url):
    download_list = []
    response = requests.get(url)
    d = pq(response.text)
    package_list = d('div').filter('#full-package-list').children('a.package-header')
    for i in range(len(package_list)):
        download_list.append('https://f-droid.org' + package_list.eq(i).attr('href'))
    partial_next = d('li').filter('.nav.next').children('a.label').attr('href')
    if partial_next != None:
        next_url = 'https://f-droid.org' + partial_next
    else:
        next_url = ''
    return download_list, next_url

In [4]:
def get_download_url(url):
    response = requests.get(url)
    d = pq(response.text)
    download_url = d('p').filter('.package-version-download').eq(0).children('a').attr('href')
    return download_url

In [5]:
import xml.sax
 
class LayoutHandler_new( xml.sax.ContentHandler ):
    result = {}
    
    def __init__(self):
        self.CurrentData = ""
        
    # 元素开始事件处理
    def startElement(self, tag, attributes):
        if '.' in tag:
            if not tag.startswith('android.'):
                if tag in LayoutHandler.result:
                    LayoutHandler.result[tag] += 1
                else:
                    LayoutHandler.result[tag] = 1
        
#     # 元素结束事件处理
#     def endElement(self, tag):
#         print('}', end = "")

class LayoutHandler( xml.sax.ContentHandler ):
    result = {}
    
    def __init__(self):
        self.CurrentData = ""
        
    # 元素开始事件处理
    def startElement(self, tag, attributes):
        if '.' in tag:
            if not tag.startswith('android.'):
                if tag not in LayoutHandler.result:
                    LayoutHandler.result[tag] = 1

# 创建一个 XMLReader
parser_new = xml.sax.make_parser()
parser = xml.sax.make_parser()
# turn off namepsaces
parser_new.setFeature(xml.sax.handler.feature_namespaces, 0)
parser.setFeature(xml.sax.handler.feature_namespaces, 0)
 
# 重写 ContextHandler
Handler_new = LayoutHandler_new()
Handler = LayoutHandler()
parser_new.setContentHandler( Handler_new )
parser.setContentHandler( Handler )

In [6]:
def analyse(rootdir):
    list =  os.listdir(rootdir)
    new = True
    for file in list:
        if not file.endswith('.xml'):
            continue
        if new:
            parser_new.parse(os.path.join(rootdir,file))
            new = False
        else:
            parser.parse(os.path.join(rootdir,file))

In [7]:
def traverse(filename):
    rootdir = filename + '/resources/res'
    try:
        list = os.listdir(rootdir)
    except:
        pass
    else:
        #遍历./resources/res/layout (layout-...) 中XML文件，进行统计
        for i in range(0,len(list)):
            if list[i].startswith('layout'):
                path = os.path.join(rootdir,list[i])
                analyse(path)

In [71]:
count = 0
successful_count = 0

url = input("Please input initial URL: ")
download_list = []
page_count = 0

result = {}

#一次最多爬取10页，300个apk
while url != '' and page_count < 10:
    print('crawling URL: ' + url)
    
    #分析网页，获得package list和下个url
    download_list, url = crawl(url)
    for package_url in download_list:
        download_url = get_download_url(package_url)
        
        #下载APK
        download_apk(download_url)
        
        #反编译
        filename = os.path.basename(download_url).split('?')[0]
        file = os.popen('jadx ' + str(filename))
        file.read()
        
        #分析第三方component
        traverse(str(filename)[:-4])
    page_count += 1
print('共尝试下载' + str(count) + '个APK，成功' + str(successful_count) + '个')

Please input initial URL: https://f-droid.org/en/packages/12/
crawling URL: https://f-droid.org/en/packages/12/
第1条url:
https://f-droid.org/repo/org.ninthfloor.copperpdf_4.apk
文件类型：application/vnd.android.package-archive
文件大小：0.4M
文件名：org.ninthfloor.copperpdf_4.apk
Succeed

第2条url:
https://f-droid.org/repo/org.billthefarmer.currency_120.apk
文件类型：application/vnd.android.package-archive
文件大小：0.46M
文件名：org.billthefarmer.currency_120.apk
Succeed

第3条url:
https://f-droid.org/repo/tyagi.shubham.customsearch_1.apk
文件类型：application/vnd.android.package-archive
文件大小：1.13M
文件名：tyagi.shubham.customsearch_1.apk
Succeed

第4条url:
https://f-droid.org/repo/com.github.cythara_19.apk
文件类型：application/vnd.android.package-archive
文件大小：1.68M
文件名：com.github.cythara_19.apk
Succeed

第5条url:
https://f-droid.org/repo/com.hasi.hasid00r_3.apk
文件类型：application/vnd.android.package-archive
文件大小：1.25M
文件名：com.hasi.hasid00r_3.apk
Succeed

第6条url:
https://f-droid.org/repo/org.mult.daap_40.apk
文件类型：application/vnd.androi

文件类型：application/vnd.android.package-archive
文件大小：8.88M
文件名：org.sufficientlysecure.viewer_2817.apk
Succeed

第50条url:
https://f-droid.org/repo/com.quaap.dodatheexploda_3.apk
文件类型：application/vnd.android.package-archive
文件大小：2.5M
文件名：com.quaap.dodatheexploda_3.apk
Succeed

第51条url:
https://f-droid.org/repo/com.nltechno.dolidroidpro_28.apk
文件类型：application/vnd.android.package-archive
文件大小：18.75M
文件名：com.nltechno.dolidroidpro_28.apk
Succeed

第52条url:
https://f-droid.org/repo/org.dolphinemu.dolphinemu_14523.apk
文件类型：application/vnd.android.package-archive
文件大小：13.76M
文件名：org.dolphinemu.dolphinemu_14523.apk
Succeed

第53条url:
https://f-droid.org/repo/org.domogik.domodroid13_33.apk
文件类型：application/vnd.android.package-archive
文件大小：3.68M
文件名：org.domogik.domodroid13_33.apk
Succeed

第54条url:
https://f-droid.org/repo/fr.mobdev.blooddonation_12.apk
文件类型：application/vnd.android.package-archive
文件大小：1.94M
文件名：fr.mobdev.blooddonation_12.apk
Succeed

第55条url:
https://f-droid.org/repo/us.bravender.andro

Succeed

第99条url:
https://f-droid.org/repo/de.vanitasvitae.enigmandroid_17.apk
文件类型：application/vnd.android.package-archive
文件大小：0.7M
文件名：de.vanitasvitae.enigmandroid_17.apk
Succeed

第100条url:
https://f-droid.org/repo/com.nutomic.ensichat_17.apk
文件类型：application/vnd.android.package-archive
文件大小：7.72M
文件名：com.nutomic.ensichat_17.apk
Succeed

第101条url:
https://f-droid.org/repo/ca.pr0ps.xposed.entrustunblocker_3.apk
文件类型：application/vnd.android.package-archive
文件大小：0.03M
文件名：ca.pr0ps.xposed.entrustunblocker_3.apk
Succeed

第102条url:
https://f-droid.org/repo/org.epstudios.epmobile_50.apk
文件类型：application/vnd.android.package-archive
文件大小：2.3M
文件名：org.epstudios.epmobile_50.apk
Succeed

第103条url:
https://f-droid.org/repo/org.jamienicol.episodes_12.apk
文件类型：application/vnd.android.package-archive
文件大小：1.49M
文件名：org.jamienicol.episodes_12.apk
Succeed

第104条url:
https://f-droid.org/repo/pl.hypeapp.episodie_2.apk
文件类型：application/vnd.android.package-archive
文件大小：5.55M
文件名：pl.hypeapp.episodie_2.apk

文件类型：application/vnd.android.package-archive
文件大小：0.4M
文件名：eu.veldsoft.fish.rings_3.apk
Succeed

第148条url:
https://f-droid.org/repo/com.workingagenda.fissure_5.apk
文件类型：application/vnd.android.package-archive
文件大小：1.81M
文件名：com.workingagenda.fissure_5.apk
Succeed

第149条url:
https://f-droid.org/repo/com.abhijitvalluri.android.fitnotifications_40.apk
文件类型：application/vnd.android.package-archive
文件大小：3.93M
文件名：com.abhijitvalluri.android.fitnotifications_40.apk
Succeed

第150条url:
https://f-droid.org/repo/com.miqote.shanawp_10.apk
文件类型：application/vnd.android.package-archive
文件大小：1.94M
文件名：com.miqote.shanawp_10.apk
Succeed

crawling URL: https://f-droid.org/en/packages/17/
第151条url:
https://f-droid.org/repo/com.simplemobiletools.flashlight_30.apk
文件类型：application/vnd.android.package-archive
文件大小：4.42M
文件名：com.simplemobiletools.flashlight_30.apk
Succeed

第152条url:
https://f-droid.org/repo/ca.cmetcalfe.xposed.flatconnectivityicons_1.apk
文件类型：application/vnd.android.package-archive
文件大小：0.12M


Succeed

第196条url:
https://f-droid.org/repo/com.tobykurien.google_news_18.apk
文件类型：application/vnd.android.package-archive
文件大小：0.07M
文件名：com.tobykurien.google_news_18.apk
Succeed

第197条url:
https://f-droid.org/repo/com.mridang.speedo_2.apk
文件类型：application/vnd.android.package-archive
文件大小：11.59M
文件名：com.mridang.speedo_2.apk
Succeed

第198条url:
https://f-droid.org/repo/com.gcstar.viewer_14.apk
文件类型：application/vnd.android.package-archive
文件大小：0.78M
文件名：com.gcstar.viewer_14.apk
Succeed

第199条url:
https://f-droid.org/repo/org.sugr.gearshift_88.apk
文件类型：application/vnd.android.package-archive
文件大小：2.48M
文件名：org.sugr.gearshift_88.apk
Succeed

第200条url:
https://f-droid.org/repo/pl.nkg.geokrety_47.apk
文件类型：application/vnd.android.package-archive
文件大小：2.49M
文件名：pl.nkg.geokrety_47.apk
Succeed

第201条url:
https://f-droid.org/repo/se.pp.mc.android.Gerberoid_10000.apk
文件类型：application/vnd.android.package-archive
文件大小：1.96M
文件名：se.pp.mc.android.Gerberoid_10000.apk
Succeed

第202条url:
https://f-droid.

文件类型：application/vnd.android.package-archive
文件大小：0.08M
文件名：ch.blinkenlights.android.vanillaplug_160.apk
Succeed

第244条url:
https://f-droid.org/repo/de.cweiske.headphoneindicator_1.apk
文件类型：application/vnd.android.package-archive
文件大小：0.01M
文件名：de.cweiske.headphoneindicator_1.apk
Succeed

第245条url:
https://f-droid.org/repo/org.katsarov.heatcalc_10102.apk
文件类型：application/vnd.android.package-archive
文件大小：1.89M
文件名：org.katsarov.heatcalc_10102.apk
Succeed

第246条url:
https://f-droid.org/repo/com.borneq.heregpslocation_1.apk
文件类型：application/vnd.android.package-archive
文件大小：0.05M
文件名：com.borneq.heregpslocation_1.apk
Succeed

第247条url:
https://f-droid.org/repo/net.damsy.soupeaucaillou.heriswap_1603042149.apk
文件类型：application/vnd.android.package-archive
文件大小：7.57M
文件名：net.damsy.soupeaucaillou.heriswap_1603042149.apk
Succeed

第248条url:
https://f-droid.org/repo/com.eneko.hexcolortimewallpaper_2.apk
文件类型：application/vnd.android.package-archive
文件大小：1.32M
文件名：com.eneko.hexcolortimewallpaper_2.apk

Succeed

第292条url:
https://f-droid.org/repo/com.jtechme.jumpgo_113.apk
文件类型：application/vnd.android.package-archive
文件大小：4.48M
文件名：com.jtechme.jumpgo_113.apk
Succeed

第293条url:
https://f-droid.org/repo/uk.co.jarofgreen.JustADamnCompass_2.apk
文件类型：application/vnd.android.package-archive
文件大小：0.02M
文件名：uk.co.jarofgreen.JustADamnCompass_2.apk
Succeed

第294条url:
https://f-droid.org/repo/com.alaskalinuxuser.justcraigslist_10.apk
文件类型：application/vnd.android.package-archive
文件大小：3.98M
文件名：com.alaskalinuxuser.justcraigslist_10.apk
Succeed

第295条url:
https://f-droid.org/repo/com.alaskalinuxuser.justnotes_11.apk
文件类型：application/vnd.android.package-archive
文件大小：1.77M
文件名：com.alaskalinuxuser.justnotes_11.apk
Succeed

第296条url:
https://f-droid.org/repo/co.pxhouse.sas_6.apk
文件类型：application/vnd.android.package-archive
文件大小：0.1M
文件名：co.pxhouse.sas_6.apk
Succeed

第297条url:
https://f-droid.org/repo/com.alaskalinuxuser.justchess_10.apk
文件类型：application/vnd.android.package-archive
文件大小：3.24M
文件名：com.al

In [29]:
list = os.listdir()
for filename in list:
    if os.path.isdir(filename):
        print(filename)
        traverse(filename)
#     if os.path.isdir(list[i]):
#         traverse(list[i])

demo
com.uberspot.a2048_23
br.com.originalsoftware.taxifonecliente.coopetaxiRecife_2018-06-05
nerd.tuxmobil.fahrplan.camp_39
com.traffar.a24game_5
demo.unaligned
.ipynb_checkpoints
info.staticfree.android.twentyfourhour_5


In [73]:
#将结果转化为dataframe便于统计。保存入CSV文件中
import pandas as pd
data = pd.DataFrame(LayoutHandler.result, index=['num']).T
data.to_csv("result_2.csv",index=True,sep=',')

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv("result_2.csv", sep=',')
data.rename(columns={'Unnamed: 0':'component'}, inplace = True)

In [9]:
dict = {}
for i in range(data.shape[0]):
    key = data.iloc[i]['component'].partition('.')[0]
    if key in dict:
        dict[key] += data.iloc[i]['num']
    else:
        dict[key] = data.iloc[i]['num']

In [11]:
LayoutHandler.result = dict

In [12]:
LayoutHandler.result

{'me': 7,
 'com': 1177,
 'de': 53,
 'br': 1,
 'nerd': 6,
 'info': 3,
 'subreddit': 1,
 'org': 347,
 'aarddict': 2,
 'androidx': 47,
 'io': 23,
 'net': 70,
 'haibison': 1,
 'dev': 18,
 'kiwi': 1,
 'li': 1,
 'in': 3,
 'gq': 1,
 'mehdi': 1,
 'rikka': 2,
 'jp': 3,
 'ch': 4,
 'im': 2,
 'calculator': 5,
 'agency': 3,
 'afzkl': 6,
 'indrora': 1,
 'souch': 1,
 'pl': 17,
 'be': 8,
 'yuku': 12,
 'se': 4,
 'filter': 7,
 'gs': 6,
 'core': 8,
 'update': 1,
 'adblocker': 1,
 'mobi': 4,
 'fr': 16,
 'no': 2,
 'uk': 28,
 'es': 6,
 'us': 8,
 'lecho': 4,
 'cz': 1,
 'ru': 3,
 'douzifly': 7,
 'eu': 7,
 'krasilnikov': 1,
 'likeanimation': 3,
 'uz': 1,
 'at': 7,
 'giraffine': 1,
 'activities': 1,
 'biz': 1,
 'nl': 2,
 'github': 9,
 'ryey': 2,
 'rehanced': 1,
 'koeln': 2,
 'it': 3,
 'mbmb5': 1,
 'community': 27,
 'cn': 1,
 'link': 3,
 'mozilla': 1,
 'ca': 4,
 'sk': 5,
 'nodomain': 2,
 'zame': 11,
 'Internal': 3,
 'defpackage': 1}